In [1]:
# pip install imbalanced-learn
# pip install lightgbm

import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from imblearn.over_sampling import SMOTE

In [2]:
DATASET_DIRECTORY = '../dataset/CICIoT2023/'

In [3]:
# Defina suas colunas de entrada X e a coluna de saída y
X_columns = [
    'flow_duration', 'Header_Length', 'Protocol Type', 'Duration',
    'Rate', 'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number',
    'rst_flag_number', 'psh_flag_number', 'ack_flag_number',
    'ece_flag_number', 'cwr_flag_number', 'ack_count',
    'syn_count', 'fin_count', 'urg_count', 'rst_count',
    'HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP',
    'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC', 'Tot sum', 'Min',
    'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number', 'Magnitue',
    'Radius', 'Covariance', 'Variance', 'Weight',
]
y_column = 'label'

In [4]:
# Obtém a lista de arquivos CSV no diretório
df_sets = [k for k in os.listdir(DATASET_DIRECTORY) if k.endswith('.csv')]
df_sets.sort()

# Divide os conjuntos de treinamento e teste
training_sets = df_sets[:int(len(df_sets) * 0.8)]
test_sets = df_sets[int(len(df_sets) * 0.8):]

In [5]:
# Inicializa o scaler fora do loop
scaler = StandardScaler()

In [6]:
# Inicializa um dicionário para mapear rótulos de classe
dict_7classes = {}
dict_7classes['DDoS-RSTFINFlood'] = 'DDoS'
dict_7classes['DDoS-PSHACK_Flood'] = 'DDoS'
dict_7classes['DDoS-SYN_Flood'] = 'DDoS'
dict_7classes['DDoS-UDP_Flood'] = 'DDoS'
dict_7classes['DDoS-TCP_Flood'] = 'DDoS'
dict_7classes['DDoS-ICMP_Flood'] = 'DDoS'
dict_7classes['DDoS-SynonymousIP_Flood'] = 'DDoS'
dict_7classes['DDoS-ACK_Fragmentation'] = 'DDoS'
dict_7classes['DDoS-UDP_Fragmentation'] = 'DDoS'
dict_7classes['DDoS-ICMP_Fragmentation'] = 'DDoS'
dict_7classes['DDoS-SlowLoris'] = 'DDoS'
dict_7classes['DDoS-HTTP_Flood'] = 'DDoS'

dict_7classes['DoS-UDP_Flood'] = 'DoS'
dict_7classes['DoS-SYN_Flood'] = 'DoS'
dict_7classes['DoS-TCP_Flood'] = 'DoS'
dict_7classes['DoS-HTTP_Flood'] = 'DoS'


dict_7classes['Mirai-greeth_flood'] = 'Mirai'
dict_7classes['Mirai-greip_flood'] = 'Mirai'
dict_7classes['Mirai-udpplain'] = 'Mirai'

dict_7classes['Recon-PingSweep'] = 'Recon'
dict_7classes['Recon-OSScan'] = 'Recon'
dict_7classes['Recon-PortScan'] = 'Recon'
dict_7classes['VulnerabilityScan'] = 'Recon'
dict_7classes['Recon-HostDiscovery'] = 'Recon'

dict_7classes['DNS_Spoofing'] = 'Spoofing'
dict_7classes['MITM-ArpSpoofing'] = 'Spoofing'

dict_7classes['BenignTraffic'] = 'Benign'

dict_7classes['BrowserHijacking'] = 'Web'
dict_7classes['Backdoor_Malware'] = 'Web'
dict_7classes['XSS'] = 'Web'
dict_7classes['Uploading_Attack'] = 'Web'
dict_7classes['SqlInjection'] = 'Web'
dict_7classes['CommandInjection'] = 'Web'


dict_7classes['DictionaryBruteForce'] = 'BruteForce'

In [8]:
# Loop de treinamento
for train_set in tqdm(training_sets):
    df_train = pd.read_csv(os.path.join(DATASET_DIRECTORY, train_set))
    df_train[X_columns] = scaler.fit_transform(df_train[X_columns])
    
    # Mapeie as classes 'Attack' e 'Benign' para 0 e 1
    df_train[y_column] = df_train[y_column].map(dict_7classes)
    
    # Aplica o SMOTE para balancear os dados
    smote = SMOTE(sampling_strategy='auto', random_state=42)
    X_resampled, y_resampled = smote.fit_resample(df_train[X_columns], df_train[y_column])
    
    lgbm_model = LGBMClassifier()  # Use o LGBMClassifier
    lgbm_model.fit(X_resampled, y_resampled)

  0%|          | 0/135 [00:00<?, ?it/s]

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.113704 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7965
[LightGBM] [Info] Number of data points in the train set: 1390216, number of used features: 40
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442


  1%|          | 1/135 [02:09<4:50:07, 129.91s/it]

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.115912 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7859
[LightGBM] [Info] Number of data points in the train set: 1275272, number of used features: 40
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442


  1%|▏         | 2/135 [04:06<4:31:13, 122.35s/it]

...

 99%|█████████▊| 133/135 [4:53:01<05:00, 150.46s/it]

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.138265 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7816
[LightGBM] [Info] Number of data points in the train set: 1398424, number of used features: 39
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442


 99%|█████████▉| 134/135 [4:55:02<02:21, 141.47s/it]

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.408135 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7924
[LightGBM] [Info] Number of data points in the train set: 1419408, number of used features: 40
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442


100%|██████████| 135/135 [4:57:56<00:00, 132.42s/it]


In [10]:
# Avaliação no conjunto de teste
y_test = []
y_pred_lgbm = []

for test_set in tqdm(test_sets):
    df_test = pd.read_csv(os.path.join(DATASET_DIRECTORY, test_set))
    df_test[X_columns] = scaler.transform(df_test[X_columns])
    
    # Mapeie as classes 'Attack' e 'Benign' para 0 e 1
    df_test[y_column] = df_test[y_column].map(dict_7classes)
    
    y_test += list(df_test[y_column].values)
    
    # Use o modelo LGBM treinado
    y_pred = list(lgbm_model.predict(df_test[X_columns]))
    y_pred_lgbm += y_pred

100%|██████████| 34/34 [04:33<00:00,  8.04s/it]


In [11]:
# Avalie o desempenho do modelo LightGBM
print("##### LightGBM (2 classes) #####")
print('accuracy_score: ', accuracy_score(y_pred_lgbm, y_test))
print('recall_score: ', recall_score(y_pred_lgbm, y_test, average='macro'))
print('precision_score: ', precision_score(y_pred_lgbm, y_test, average='macro'))
print('f1_score: ', f1_score(y_pred_lgbm, y_test, average='macro'))

##### LightGBM (2 classes) #####
accuracy_score:  0.9946303544016384
recall_score:  0.8227339023903931
precision_score:  0.8222236160490004
f1_score:  0.8213012544054076


In [12]:
from sklearn.metrics import accuracy_score, classification_report

# Avalie o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred_lgbm)
report = classification_report(y_test, y_pred_lgbm)

print("Acurácia:", accuracy)
print("Relatório de Classificação:")
print(report)

Acurácia: 0.9946303544016384
Relatório de Classificação:
              precision    recall  f1-score   support

      Benign       0.93      0.92      0.93    243322
  BruteForce       0.60      0.49      0.54      2983
        DDoS       1.00      1.00      1.00   7526151
         DoS       1.00      1.00      1.00   1792167
       Mirai       1.00      1.00      1.00    583677
       Recon       0.82      0.84      0.83     78630
    Spoofing       0.84      0.86      0.85    107798
         Web       0.39      0.47      0.42      5433

    accuracy                           0.99  10340161
   macro avg       0.82      0.82      0.82  10340161
weighted avg       0.99      0.99      0.99  10340161



In [ ]:
#Salvando o Modelo Treinado
import joblib
# Salve o modelo treinado em um arquivo
joblib.dump(lgbm_model, 'modelo_lgbm.pkl')

['modelo_lgbm.pkl']